# Multiscale PV Recycling MILP

In [1]:
__author__ = "Natasha Jane Chrisandina, Funda Iseri"
__copyright__ = "Copyright 2023, Multi-parametric Optimization & Control Lab"
__credits__ = ["Natasha Jane Chrisandina", "Funda Iseri", "Efstratios N. Pistikopoulos"]
__license__ = "MIT"
__version__ = "1.0.0"
__maintainer__ = "Natasha Jane Chrisandina"
__email__ = "nchrisandina@tamu.edu"
__status__ = "Production"


## Problem Statement

The following case study considers one type of solar PV panel (crystalline silicon, c-Si) that is to be recycled into separate components. The goal is to maximize economic profit obtained from the recycling process.

Data source: https://www.sciencedirect.com/science/article/pii/S0927024822000162

Module imports

In [3]:
import pandas 
import numpy

from src.energiapy.components.temporal_scale import TemporalScale
from src.energiapy.components.resource import Resource
from src.energiapy.components.process import Process
from src.energiapy.components.location import Location
from src.energiapy.components.transport import Transport
from src.energiapy.components.network import Network
from src.energiapy.components.scenario import Scenario
from src.energiapy.components.result import Result 
from src.energiapy.model.formulate import formulate, Constraints, Objective
from src.energiapy.plot import plot_results, plot_scenario
from src.energiapy.plot.plot_results import CostY, CostX
from src.energiapy.model.solve import solve
import matplotlib.pyplot as plt
from matplotlib import rc
from itertools import product

### Initialize model

Define temporal scales at different levels:
- 0 is annual, with 1 discretization
- 1 is daily, with 365 discretization

In [23]:
scales = TemporalScale(discretization_list=[1, 365])

Define resources, which are things to be consumed, produced, stored, or sold. In this case, the feedstock is silicon PV and the products are the different major recyclable materials. We will assume that *glass* is a product in demand, whereas everything else can simply be sold.

In [24]:
SiPV_dummy = Resource(name = 'sipv_dummy', label='Silicon PV Dummy', price=50, basis = 'module', cons_max=130000)
SiPV = Resource(name = 'sipv', label='Silicon Photovoltaic', price=50, basis = 'module', cons_max=130000)
glass = Resource(name = 'glass', label='Recycled glass', revenue=100, basis = 'kg', cons_max=10**6)
aluminum = Resource(name = 'al', label='Recycled aluminum', revenue=100, basis = 'kg', sell=True)
polymer = Resource(name = 'polymer', label='Recycled polymer', revenue=100, basis = 'kg', sell=True)
copper = Resource(name = 'cu', label='Recycled copper', revenue=100, basis = 'kg', sell=True)
silver = Resource(name = 'ag', label='Recycled silver', revenue=100, basis = 'kg', sell=True)
silicon = Resource(name = 'si', label='Recycled silicon', revenue=100, basis = 'kg', sell=True)
glass_dummy = Resource(name = 'glass_dummy', label='Recycled glass Dummy', revenue=100, basis = 'kg', demand=True)


Define processes, which are the different potential recycling processes. There are three types of process: FRELP, ASU, and Hybrid, and each process has two separate max production level to represent large-scale vs small-scale recycling.

The "collection" process is essentially a dummy process to represent PVs being shipped off from a collection center to recycling center locations.

In [25]:
collection = Process(name = 'collection', label='dummy process', conversion = {SiPV_dummy: -1, SiPV: 1}, prod_max = 130000)
glass_collect = Process(name = 'glass_collection', label='dummy process for product', conversion = {glass: -1, glass_dummy: 1}, prod_max = 10**6)


FRELP = Process(name = 'FRELP', label='FRELP recycling', conversion = {SiPV: -1, glass: 13.5975, polymer: 2.22, aluminum: 1.65501, silicon: 0.53835, copper: 0.17945, silver: 0.001739}, prod_max = 1300, capex = 4, vopex = 8)
ASU = Process(name = 'ASU', label='ASU recycling', conversion = {SiPV: -1, glass: 13.73625, polymer: 2.22, aluminum: 1.5651, silicon: 0.4995, copper: 0.15355, silver: 0.001369}, prod_max = 1300, capex = 17, vopex = 9)
hybrid = Process(name = 'hybrid', label='Hybrid recycling', conversion = {SiPV: -1, glass: 13.5975, polymer: 2.22, aluminum: 1.65501, silicon: 0.53835, copper: 0.15355, silver: 0.001369}, prod_max = 1300, capex = 3, vopex = 7)


FRELP_big = Process(name = 'FRELP_big', label='big FRELP recycling', conversion = {SiPV: -1, glass: 13.5975, polymer: 2.22, aluminum: 1.65501, silicon: 0.53835, copper: 0.17945, silver: 0.001739}, prod_max = 13000, capex = 3, vopex = 5)
ASU_big = Process(name = 'ASU_big', label='big ASU recycling', conversion = {SiPV: -1, glass: 13.73625, polymer: 2.22, aluminum: 1.5651, silicon: 0.4995, copper: 0.15355, silver: 0.001369}, prod_max = 13000, capex = 8, vopex = 6)
hybrid_big = Process(name = 'hybrid_big', label='big Hybrid recycling', conversion = {SiPV: -1, glass: 13.5975, polymer: 2.22, aluminum: 1.65501, silicon: 0.53835, copper: 0.15355, silver: 0.001369}, prod_max = 13000, capex = 3, vopex = 4)


Define locations

In [26]:
cc1 = Location(name='cc1', label = 'Collection Center 1', scales = scales, processes = {collection})
cc2 = Location(name='cc2', label = 'Collection Center 2', scales = scales, processes = {collection})


rc1 = Location(name='rc1', label = 'Recycling Center 1', scales = scales, processes = {FRELP, ASU, hybrid, FRELP_big, ASU_big, hybrid_big})

gc1 = Location(name='gc1', label = 'Glass Recycling Center 1', scales = scales, processes= {glass_collect})


Define transportation modes

In [36]:
Truck = Transport(name = 'Truck', resources = {SiPV, SiPV_dummy, glass, aluminum, polymer, copper, silver, silicon, glass_dummy}, trans_max = 10**6, trans_cost = 0.01, label = 'Generic truck for everything')

distance_matrix = [
    [0, 100, 100, 100],
    [100, 0, 100, 100],
    [100, 100, 0, 100],
    [100, 100, 100, 0]
]

transport_matrix = [
    [[], [Truck], [Truck],[Truck]],
    [[Truck],[], [Truck], [Truck]],
    [[Truck], [Truck], [], [Truck]],
    [[Truck], [Truck],[Truck],[]]
]

network = Network(name = 'Network', label= 'Generic network', source_locations = [cc1, cc2, rc1, gc1], sink_locations = [gc1], distance_matrix = distance_matrix, transport_matrix= transport_matrix)

Define scenario

In [37]:
genericScenario = Scenario(name = 'genericScenario', network = network, scales= scales, demand={gc1: {glass_dummy: 1000}}, label = 'genericScenario', network_scale_level = 0, purchase_scale_level = 1, scheduling_scale_level = 1, demand_scale_level = 1, expenditure_scale_level = 0)

MILP formulation

In [38]:
milp_demand = formulate(scenario = genericScenario, constraints={Constraints.INVENTORY,Constraints.COST, Constraints.PRODUCTION, Constraints.RESOURCE_BALANCE, Constraints.TRANSPORT}, objective=Objective.MAX_DISCHARGE, objective_resource=glass_dummy)

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint location fopex
constraint location vopex
constraint location incidental
constraint network capex
constraint network fopex
constraint network vopex
constraint network incidental
constraint nameplate inventory
constraint storage max
constraint storage min
constraint production mode
constraint nameplate production
constraint production max
constraint production min
constraint inventory balance
constraint resource consumption
constraint resource purchase
constraint location production
constraint location discharge
constraint location consumption
constraint location purchase
constraint network production
constraint network discharge
constraint network consumption
constraint network purchase
constraint transport export
constraint transport import
constraint transport exp UB
constraint transport imp UB
constraint transport balance
constraint transport i

In [40]:
results = solve(scenario = genericScenario, instance=milp_demand, solver='gurobi', print_solversteps=True, name = 'results')

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter LogFile to C:\Users\NCHRIS~1\AppData\Local\Temp\tmp2b_ci4zh.log
   Prev:   Default: 
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 116749 rows, 120371 columns and 325809 nonzeros
Model fingerprint: 0xbebbbce4
Variable types: 120367 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+06]
Presolve removed 116749 rows and 120371 columns
Presolve time: 0.11s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.14 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 3.55078e+09 

Optimal solution found (tolerance 1.00e-04)
Be

KeyboardInterrupt: 